In [1]:
import pandas as pd
import numpy as np
import os
import sys
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
import torch.nn as nn
np.random.seed(42)
import torch
torch.manual_seed(42)
from model.model_lstm import train,predict
from scipy.stats import spearmanr
import os

In [ ]:
raw_data = pd.read_hdf("/home/laiminzhi/data/xydata/xy_data_3355.h5")

In [7]:

def preprocess(data):
    #去掉缺失值
    data = data.replace([np.inf,-np.inf],np.nan)
    data = data.replace(np.nan,0)
    #筛选出universe中的数据
    data =data[data['univ_tradable'] == 1]
    #去掉涨跌停数据
    data = data[data['ud_limit_h2'] == 0]
    #把无关的列删掉
    irrelevant_cols = data.columns[data.columns.str.startswith('u')].to_list() + data.columns[data.columns.str.startswith('y')].to_list()
    irrelevant_cols.remove('y1_label')
    data = data.drop(columns = irrelevant_cols)

    feature_data = data.loc[:,data.columns.str.startswith('x')]
    label = data.loc[:,'y1_label'].to_frame()

    return feature_data,label

feature_data,label = preprocess(raw_data)   

In [8]:
# 定义timestep和feature数
timestep = 30
num_features = feature_data.shape[1]

# 获取独立的code列表和date列表
codes = feature_data.index.get_level_values('code').unique()
dates = feature_data.index.get_level_values('date').unique()

# 初始化一个列表来存储结果
df = feature_data.reset_index()
result = []

# 对每个code应用滑动窗口
for code in codes:
    df_code = df.loc[df['code'] == code]
    print(df_code.shape)
    
    

# 将结果转换为numpy数组并重塑为所需形状
result_np = np.array(result)
result_np = result_np.reshape(-1, len(codes), timestep, num_features)

print(result_np.shape)  # 输出结果的shape



(1214, 244)
(1212, 244)
(1185, 244)
(1207, 244)
(1180, 244)
(1202, 244)
(1201, 244)
(1199, 244)
(1187, 244)
(1198, 244)
(1179, 244)
(1195, 244)
(1199, 244)
(1200, 244)
(1193, 244)
(1200, 244)
(1202, 244)
(1215, 244)
(1202, 244)
(1190, 244)
(1205, 244)
(1178, 244)
(1209, 244)
(1190, 244)
(1203, 244)
(1198, 244)
(1204, 244)
(1206, 244)
(1193, 244)
(1204, 244)
(1190, 244)
(1178, 244)
(1204, 244)
(1210, 244)
(1202, 244)
(1205, 244)
(1207, 244)
(1197, 244)
(1193, 244)


KeyboardInterrupt: 

In [27]:
result = np.array(result)
print(result[0].shape,len(result))

(1185, 30, 244) 1


In [31]:
print(result[0][-1,:,:])

[['20221028' '000001.SZ' 0.2052833984105299 ... 0.00620713106425107
  0.12264788005487444 0.12264788005487444]
 ['20221031' '000001.SZ' 0.19928509591155474 ... 0.03983291281766315
  0.02867526722081012 0.02867526722081012]
 ['20221101' '000001.SZ' 0.18753653269801393 ... 0.04312819726899394
  -0.01867141461139801 0.01867141461139801]
 ...
 ['20221207' '000001.SZ' 0.16321831387791946 ... 0.0071445269364235455
  -0.015214145285323548 0.015214145285323548]
 ['20221208' '000001.SZ' 0.13768191286473208 ... 0.0063160241530270484
  -0.06753685151770103 0.06753685151770103]
 ['20221209' '000001.SZ' 0.20444581392518596 ... 0.04695957336968256
  -0.19212144359207156 0.19212144359207156]]


In [ ]:
print(dates.to_list()[-1])